In [1]:
import os

import numpy as np
import pandas as pd
import xlrd

pd.set_option("display.max_rows", 50000)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
fundamentals = pd.read_csv("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Fundamentals_Full.csv", na_values=' ')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,25,29,916,917,919,920,926,935,937,940,941,942,943,945,955,956,959,962,965,970,978,981,984,989,992,993,997,998,1008,1009,1014,1015,1016,1017,1018,1019,1020,1021,1024,1025,1028,1029,1031,1033,1034,1035,1036,1040,1041,1042,1052,1053,1054,1055,1056,1057,1058,1059,1066,1071,1072,1074,1075,1076,1077,1081,1084,1091,1094,1096,1098,1101,1102,1103,1105,1106,1107,1108,1116,1117,1119,1120,1121,1122,1123,1124,1125,1735,1736,1737,1742,1756,1757,1762) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
securities = pd.read_csv("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Securities_Full.csv", na_values=' ', parse_dates=["datadate"])

In [5]:
fund_settlement = pd.read_csv("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Fund_Settlement.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,25,29,916,917,919,920,926,935,937,940,941,942,943,945,955,956,959,962,965,970,978,981,984,989,992,993,997,998,1008,1009,1014,1015,1016,1017,1018,1019,1020,1021,1024,1025,1028,1029,1031,1033,1034,1035,1036,1040,1041,1042,1052,1053,1054,1055,1056,1057,1058,1059,1066,1071,1072,1074,1075,1076,1077,1081,1084,1091,1094,1096,1098,1101,1102,1103,1105,1106,1107,1108,1116,1117,1119,1120,1121,1122,1123,1124,1125,1735,1736,1737,1742,1756,1757,1762) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
def excel_to_dictionary(sheetname, filename="/content/drive/MyDrive/Fiverr/PROJECT_MIMI/DataDictionary.xlsx"):
    abbreviation_dict = dict()
    
    abbreviation = xlrd.open_workbook(os.path.abspath(filename))
    abbreviation_sheet = abbreviation.sheet_by_name(sheetname)
    
    for i in range(1, abbreviation_sheet.nrows):
        abb = abbreviation_sheet.cell(i, 0).value
        abb_definition = abbreviation_sheet.cell(i, 1).value
        abbreviation_dict[abb] = abb_definition
        
    return abbreviation_dict

In [7]:
securities_dict = excel_to_dictionary(sheetname="Securities")
fundamentals_dict = excel_to_dictionary(sheetname="Fundamentals")

In [8]:
cols = [col.lower() for col in ["Dvpspm", "Dvpsxm", "Dvrate", "Csfsm", "Cahtrm", "Curcdm"]]

abbreviation_dict = dict()
    
abbreviation = xlrd.open_workbook(os.path.abspath("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/DataDictionary.xlsx"))
abbreviation_sheet = abbreviation.sheet_by_name("Securities")
    
for i in range(1, abbreviation_sheet.nrows):
  abb = abbreviation_sheet.cell(i, 0).value
  abb_definition = abbreviation_sheet.cell(i, 1).value
  abbreviation_dict[abb] = abb_definition

for col in cols:
  for key, value in abbreviation_dict.items():
    if key == col:
      print("{}: {}".format(key, value))

dvpspm: Dividends per Share - Pay Date - Monthly
dvpsxm: Dividends per Share - Ex Date - Monthly
dvrate: Dividend Rate - Monthly
csfsm: Common Stock Float Shares - Canada
curcdm: ISO Currency Code - Monthly


In [9]:
def rename_columns(df, df_dict):
  df.rename(columns=df_dict, inplace=True)
  df.columns = [column.lower().replace(" ", "_") for column in df.columns]
  return df

In [10]:
securities = rename_columns(securities, securities_dict)
fundamentals = rename_columns(fundamentals, fundamentals_dict)

In [11]:
securities["year"] = securities['data_date_-_security_monthly'].apply(lambda x: x.strftime("%Y"))

In [12]:
securities[securities["global_company_key"] == 316056][["global_company_key", "data_date_-_security_monthly", "year", "dividend_rate_-_monthly", 'dividends_per_share_-_ex_date_-_monthly', 'dividends_per_share_-_pay_date_-_monthly', 'common_stock_float_shares_-_canada', 'iso_currency_code_-_monthly']].tail(10)

,global_company_key,data_date_-_security_monthly,year,dividend_rate_-_monthly,dividends_per_share_-_ex_date_-_monthly,dividends_per_share_-_pay_date_-_monthly,common_stock_float_shares_-_canada,iso_currency_code_-_monthly
915374,316056,2013-11-30,2013,NaN,NaN,NaN,NaN,USD
915375,316056,2013-12-31,2013,NaN,NaN,NaN,NaN,USD
915376,316056,2014-01-31,2014,NaN,NaN,NaN,NaN,USD
915377,316056,2014-02-28,2014,NaN,NaN,NaN,NaN,USD
915378,316056,2014-03-31,2014,0.32,0.08,0.08,NaN,USD
915379,316056,2014-04-30,2014,0.32,NaN,NaN,NaN,USD
915380,316056,2014-05-31,2014,0.32,NaN,NaN,NaN,USD
915381,316056,2014-06-30,2014,0.32,0.08,0.08,NaN,USD


In [13]:
securities.shape

(915386, 56)

In [14]:
securities = securities.groupby(["global_company_key", "year", "iso_currency_code_-_monthly"])[["dividend_rate_-_monthly", "dividends_per_share_-_ex_date_-_monthly", 'dividends_per_share_-_pay_date_-_monthly', 'common_stock_float_shares_-_canada']].agg("sum").reset_index()

In [15]:
securities.shape

(90109, 7)

In [16]:
securities = securities.groupby(["global_company_key", "iso_currency_code_-_monthly"])[["dividend_rate_-_monthly", "dividends_per_share_-_ex_date_-_monthly", 'dividends_per_share_-_pay_date_-_monthly', 'common_stock_float_shares_-_canada']].agg("mean").reset_index()

In [17]:
securities.shape

(22463, 6)

In [18]:
fund_settlement["sued"].unique()

array(['no', 'yes'], dtype=object)

In [19]:
fund_security = pd.merge(fund_settlement, securities, on="global_company_key")

In [20]:
fund_security[fund_security["global_company_key"] == 316056][["global_company_key", "dividend_rate_-_monthly", 'dividends_per_share_-_ex_date_-_monthly', 'dividends_per_share_-_pay_date_-_monthly', 'common_stock_float_shares_-_canada', 'iso_currency_code_-_monthly', "settlementamount", "sued", "ticker_symbol"]].tail(10)

,global_company_key,dividend_rate_-_monthly,dividends_per_share_-_ex_date_-_monthly,dividends_per_share_-_pay_date_-_monthly,common_stock_float_shares_-_canada,iso_currency_code_-_monthly,settlementamount,sued,ticker_symbol
60443,316056,0.64,0.08,0.08,0.0,USD,NaN,no,ALLE
60444,316056,0.64,0.08,0.08,0.0,USD,NaN,no,ALLE
60445,316056,0.64,0.08,0.08,0.0,USD,NaN,no,ALLE
60446,316056,0.64,0.08,0.08,0.0,USD,NaN,no,ALLE
60447,316056,0.64,0.08,0.08,0.0,USD,NaN,no,ALLE
60448,316056,0.64,0.08,0.08,0.0,USD,NaN,no,ALLE
60449,316056,0.64,0.08,0.08,0.0,USD,NaN,no,ALLE
60450,316056,0.64,0.08,0.08,0.0,USD,NaN,no,ALLE


In [21]:
fund_security.to_csv("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Fund_Settlement_Security.csv", index=False)

In [22]:
securities[securities["iso_currency_code_-_monthly"] == "CAD"].loc[21698]

global_company_key                            198955
iso_currency_code_-_monthly                      CAD
dividend_rate_-_monthly                        5.876
dividends_per_share_-_ex_date_-_monthly     0.501267
dividends_per_share_-_pay_date_-_monthly    0.475587
common_stock_float_shares_-_canada                 0
Name: 21698, dtype: object

In [23]:
fund_security.shape

(60453, 1775)